In [104]:
import json
import os
import geojson

# Preparation

In [105]:
max_x_tile = 40 
max_y_tile = 40 

# Format should always be "XXXX"
all_tiles = ["{:04d}".format(number) for number in range(max_x_tile*max_y_tile)]

#print(os.getcwd())
#print(os.listdir('40_40/GeoJSON'))

# Functions

In [106]:
def find_site_types(sitetype: str) -> int:
    '''
    Given a json, find the int_sitetype_limes and output a classifer as either 0, 1, or 2
    0: Bombentrichter
    1: Grabhügel
    2: Meilerpodium
    Input: str
    Output: int
    '''
    if sitetype == 'Bombentrichter':
        return 0
    elif sitetype == 'Grabhügel':
        return 1
    elif sitetype == 'Meilerpodium':
        return 2
    else:
        raise Exception("Did not find Bombentrichter, Grabhügel, or Meilerpodium")

In [107]:
def gen_x_y_points(feature: list):
    '''
    Takes a feature and returns a list of the x and y coordinates aswell as all points
    Input: list
    Output: list, list, and list
    '''
    points = feature['geometry']['coordinates'][0] 
    x = [p[0] for p in points]
    y = [p[1] for p in points]
    return x, y, points

In [108]:
def calc_width(x: list) -> float:
    '''
    Given a list of all x values, identifies the width of the feature
    Input: list
    Output: float
    '''
    return max(x) - min(x)

In [109]:
def calc_height(y: list) -> float:
    '''
    Given a list of all y values, identifies the height of the feature
    Input: list
    Output: float
    '''
    return max(y) - min(y)

In [110]:
def find_parameters(feature: list) -> list:
    '''
    Takes a single feature, extracts the necessary information, and returns a list of paramters.
    The parameters are in the specified order:
    [class, x_center, y_center, width, height]
    Input: list
    Output: list
    '''
    parameters = []
    
    x, y, points = gen_x_y_points(feature)
    x_cen, y_cen = ( sum(x) / len(points), sum(y) / len(points))
    
    parameters.append(find_site_types(feature['properties']['int_sitetype_limes']))
    parameters.append(x_cen)
    parameters.append(y_cen)
    parameters.append(calc_width(x))
    parameters.append(calc_height(y))
    return parameters

# Get Parameters

In [113]:
base_src_path = "40_40/GeoJSON"
base_target_path = "40_40/TXT"

for i in range(max_x_tile*max_y_tile):
    
    file_name_geo = all_tiles[i] + ".geojson"
    src_path = os.path.join(base_path, file_name_geo)
    #print(src_path)
    
    # Define target location e.g.'40_40/GeoJSON/0001.txt'
    file_name_txt = all_tiles[i] + ".txt"
    target_path = os.path.join(base_target_path, file_name_txt)
    #print(target_path)
    
    try:
        f = open(src_path)
        json_file = geojson.load(f)
    
    except:
        print(file_name + " has no features")
        continue
        
    #print(src_path)
    #print(json_file)
    
    all_my_parameters = []

    try:
        for j in range(len(json_file["features"])):
            #print("in")
        
            all_my_parameters.append(find_parameters(json_file['features'][j]))   # Add for each feature the parameters that YOLOv5 needs
    
    
            print("H" + type(all_my_paraneters))
    except:
        print("Nothing detected in this image: " + file_name_geo)
              
              
              
    # TODO: Save each detection as one line in the txt file

0038.geojson has no features
Nothing detected in this image: 0038.geojson
Nothing detected in this image: 0039.geojson
0038.geojson has no features
Nothing detected in this image: 0078.geojson
Nothing detected in this image: 0079.geojson
0038.geojson has no features
Nothing detected in this image: 0109.geojson
Nothing detected in this image: 0110.geojson
Nothing detected in this image: 0111.geojson
0038.geojson has no features
Nothing detected in this image: 0148.geojson
Nothing detected in this image: 0149.geojson
Nothing detected in this image: 0150.geojson
Nothing detected in this image: 0151.geojson
0038.geojson has no features
Nothing detected in this image: 0189.geojson
Nothing detected in this image: 0190.geojson
Nothing detected in this image: 0191.geojson
Nothing detected in this image: 0192.geojson
0038.geojson has no features
Nothing detected in this image: 0229.geojson
Nothing detected in this image: 0230.geojson
Nothing detected in this image: 0231.geojson
Nothing detected